### KNN算法原理
1. 首先，计算测试样本与每个训练样本的距离（如欧氏距离或曼哈顿距离，这里采用欧氏距离）；
2. 其次，设置超参数K，选出离该测试样本最近的K个训练样本；
3. 最后，这K个测试样本当中出现最多的类别作为该测试样本的分类。

如图所示，绿色的为测试样本，当k取3时，该样本就属于红色类；当k取5时，就属于蓝色类了。所以k值的选择很大程度影响着该算法的结果，通常k的取值不大于20。
<img src="KNN.png",width=200,height=600>

#### KNN算法伪代码：
* 计算测试样本与所有训练样本的距离
* 对距离进行升序排序，取前k个
* 计算k个样本中最多的分类

### 0.导入相关库

In [1]:
import numpy as np
import operator
import pandas as pd

### 1.解析数据

In [2]:
def file2matrix(filename):
    Olddata=pd.read_csv(open(filename),sep=',',header=None)    # 读入CSV文件
    data=Olddata.drop(0)                     # 删除第一行表头
    data=data.drop(0,axis=1)                 # 删除第一列index
    data=data.sample(frac = 1)               # 对数据进行随机打乱
    returnMat=data[[i for i in range(1,14)]].values
    returnMat=returnMat.astype(np.float32)   # 数据类型转化
    classLabelVector=data[14].values
    classLabelVector=classLabelVector.astype(np.int32)
    return returnMat,classLabelVector

In [3]:
X,y=file2matrix('wine.csv')

In [4]:
X,y

(array([[1.311e+01, 1.010e+00, 1.700e+00, ..., 1.120e+00, 3.180e+00,
         5.020e+02],
        [1.369e+01, 3.260e+00, 2.540e+00, ..., 9.600e-01, 1.820e+00,
         6.800e+02],
        [1.208e+01, 1.330e+00, 2.300e+00, ..., 1.070e+00, 3.210e+00,
         6.250e+02],
        ...,
        [1.284e+01, 2.960e+00, 2.610e+00, ..., 8.900e-01, 2.150e+00,
         5.900e+02],
        [1.419e+01, 1.590e+00, 2.480e+00, ..., 1.230e+00, 2.820e+00,
         1.680e+03],
        [1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03]], dtype=float32),
 array([2, 3, 2, 1, 2, 1, 1, 3, 1, 3, 2, 3, 2, 1, 2, 2, 1, 2, 2, 2, 3, 1,
        3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 3, 2, 1, 1, 3, 1, 3,
        3, 2, 1, 2, 1, 1, 1, 3, 2, 2, 2, 2, 3, 2, 1, 2, 2, 3, 3, 2, 2, 2,
        2, 3, 1, 2, 2, 2, 3, 1, 2, 1, 1, 2, 2, 3, 2, 3, 3, 3, 1, 2, 1, 2,
        2, 3, 3, 1, 2, 3, 1, 2, 3, 1, 2, 2, 3, 1, 2, 3, 2, 2, 3, 2, 3, 2,
        2, 1, 2, 2, 2, 3, 2, 1, 1, 3, 2, 3, 1, 1, 1, 3, 2, 1

### 2.归一化
由于特征间的数值差别较大，在计算距离时，数值大的属性会对结果产生更大的影响，这里需要对数据进行归一化：new = (old-min)/(max-min)。

In [5]:
def autoNorm(dataSet):
    minval=dataSet.min(0)   # 0轴，即按列
    maxval=dataSet.max(0)
    ranges=maxval - minval
    normDateSet=np.zeros(np.shape(dataSet))  # 构造和数据一样大的零矩阵
    m=dataSet.shape[0]
    normDataSet=dataSet-np.tile(minval,(m,1))
    normDataSet=normDataSet/np.tile(ranges,(m,1))
    return normDataSet,ranges,minval

In [6]:
# np.tile()的用法示例
a=np.array([7,8,9])
np.tile(a,(3,1))

array([[7, 8, 9],
       [7, 8, 9],
       [7, 8, 9]])

In [7]:
np.tile(a,(3,2))

array([[7, 8, 9, 7, 8, 9],
       [7, 8, 9, 7, 8, 9],
       [7, 8, 9, 7, 8, 9]])

In [8]:
new_X,ranges,minval=autoNorm(X)
new_X

array([[0.54736835, 0.05335968, 0.1818182 , ..., 0.5203252 , 0.6996337 ,
        0.15977176],
       [0.6999999 , 0.49802366, 0.631016  , ..., 0.3902439 , 0.20146522,
        0.28673324],
       [0.27631584, 0.11660079, 0.50267375, ..., 0.47967482, 0.7106227 ,
        0.24750356],
       ...,
       [0.4763159 , 0.43873516, 0.6684491 , ..., 0.3333333 , 0.32234436,
        0.22253923],
       [0.83157885, 0.16798419, 0.5989305 , ..., 0.6097561 , 0.56776553,
        1.        ],
       [0.84210515, 0.1916996 , 0.57219255, ..., 0.4552845 , 0.970696  ,
        0.5613409 ]], dtype=float32)

### 3.KNN算法
* inX为训练数据；dataSet为测试数据，labels为类别标签；k为取值
* 距离采用欧氏距离
* 对计算的距离进行索引排序（argsort），然后对字典进行排序，获取样本数最多的分类

In [9]:
def classify(inX,dataSet,labels,k):
    dataSize=dataSet.shape[0]
    diffMat=np.tile(inX,(dataSize,1))-dataSet
    sqdiffMat=diffMat**2
    sqDistance=sqdiffMat.sum(axis=1)
    distance=sqDistance**0.5
    sortedDist=distance.argsort()    # 对计算的距离进行索引排序
    classCount={}
    for i in range(k):
        voteIlable=labels[sortedDist[i]]
        classCount[voteIlable]=classCount.get(voteIlable,0)+1
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

### 4.对分类器进行测试
这里选择前10%数据（r=0.1）做为测试样本，进行分类器的测试。

In [10]:
def test():
    r=0.1
    X,y=file2matrix('wine.csv')
    new_X,ranges,minval=autoNorm(X)
    m=new_X.shape[0]
    numTestVecs=int(m*r)
    error=0.0
    for i in range(numTestVecs):
        result=classify(new_X[i,:],new_X[numTestVecs:m,:],y[numTestVecs:m],3)
        print('分类结果：%d,真实数据：%d' % (result,y[i]))
        if(result !=y[i]):
            error=error+1.0
    print('错误率 %f' % (error/float(numTestVecs)))

In [11]:
test()

分类结果：1,真实数据：1
分类结果：2,真实数据：2
分类结果：2,真实数据：2
分类结果：3,真实数据：3
分类结果：3,真实数据：2
分类结果：1,真实数据：1
分类结果：1,真实数据：1
分类结果：3,真实数据：3
分类结果：1,真实数据：1
分类结果：1,真实数据：2
分类结果：2,真实数据：2
分类结果：1,真实数据：1
分类结果：1,真实数据：1
分类结果：3,真实数据：3
分类结果：1,真实数据：1
分类结果：2,真实数据：2
分类结果：2,真实数据：2
错误率 0.117647


#### 测试系统
给出一组测试数据：12.11,1.5,2.65,26.5,118,2.5,1.28,0.26,1.56,7.1,0.61,1.33,420

In [12]:
def system():
    style=['拉菲','威士忌','青岛啤酒']
    wineinput=input('请输入酒的参数：')
    winedata=np.array([float(n) for n in wineinput.split(',')])
    X, y = file2matrix('wine.csv')
    new_X, ranges, minval = autoNorm(X)
    result = classify((winedata - minval)/ranges, new_X, y, 3)
    print('你好！')
    print('这瓶酒是：',style[result-1])

In [15]:
system()

请输入酒的参数：14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065
你好！
这瓶酒是： 拉菲


### 5.算法优缺点
* 优点：精度高，对异常值不敏感
* 缺点：计算复杂（每个测试样本都要与训练样本继续距离计算）